In [546]:
import math 
import random

class Node():
    def __init__(self, value, _backward=None, parents=[]):
        self.value = value
        self.grad = 0
        self._backward = _backward
        self.parents = parents
    
    def __add__(self, other):
        other = other if isinstance(other, Node) else Node(other)
        value_add = self.value + other.value
        out = Node(value_add, parents=[self, other])

        def _backward():
            self.grad = 1 * out.grad # c = a + b => dc/da = 1
            other.grad = 1 * out.grad # c = a + b => dc/db = 1
        out._backward = _backward
        return out

    def __radd__(self,other):
        return self + other

    def __sub__(self, other):
        other = other if isinstance(other, Node) else Node(other)
        value_sub = self.value - other.value
        out = Node(value_sub, parents=[self, other])

        def _backward():
            self.grad = 1 * out.grad # c = a + b => dc/da = 1
            other.grad = -1 * out.grad # c = a + b => dc/db = 1
        out._backward = _backward
        return out

    def __rsub__(self,other):
        return self - other

    def __mul__(self, other):
        other = other if isinstance(other, Node) else Node(other)
        value_mul = self.value * other.value
        out = Node(value_mul, parents=[self, other])

        def _backward():
            self.grad = other.value * out.grad # c = a * b => dc/da = b
            other.grad = self.value * out.grad # c = a * b => dc/db = a
        out._backward = _backward
        return out
    
    def __rmul__(self,other):
        return self * other

    def __pow__(self, other):
        value_pow = self.value ** other
        out = Node(value_pow, parents=[self,])

        def _backward():
            self.grad = other * value_pow ** (other - 1) * out.grad
        out._backward = _backward
        return out

    def sigmoid(self):
        value_sigmoid = 1 / (1 + math.exp(-self.value))
        out = Node(value_sigmoid, parents=[self])
        
        def _backward():
            self.grad += value_sigmoid * (1 - value_sigmoid) * out.grad
        out._backward = _backward
        return out

    def __repr__(self):
        return f"Node( value={self.value}, grad={self.grad})"

    # BFS algorithm to perform backpropagation
    def backward(self):
        self.grad = 1

        visited = [self]
        queue = [self]

        while queue != []:
            node = queue.pop(0)        
            for parent in node.parents:
                node._backward()
                visited.append(parent)
                queue.append(parent)


In [547]:
class Neuron():
    def __init__(self, n_weights):
        self.weights = [Node(random.random()) for _ in range(n_weights)]
        self.bias = Node(random.random())
        self.parameters = self.weights + [self.bias]
    
    def __call__(self, data):
        sum_value = sum( (w*x for w,x in zip(self.weights, data)), self.bias)
        return sum_value.sigmoid()

In [562]:
# Fixed data set
n_samples = 10

x_0 = [random.random() for _ in range(n_samples)]
x_1 = [100*(0.5 - random.random()) for _ in range(n_samples)]
x = list(zip(x_0, x_1))

y_true = [1 if i[1]>0 else 0 for i in x ]

In [563]:
n = Neuron(2)

In [567]:
n_epoch = 1000
learning_rate = 0.05

for i in range(n_epoch):
    # forward pass
    y_pred = [n(x) for x in x]
    y_loss = sum((y_pred_i - y_true_i)**2 for y_pred_i,y_true_i in zip(y_pred, y_true))*(1/n_samples)

    # backward
    for p in n.parameters:
        p.grad = 0
    y_loss.backward()

    # update
    for p in n.parameters:
        p.value += -learning_rate*p.grad*(1/n_samples)

    if i%100 == 0:
        print(y_loss.value)


0.014427728599124426
0.014222436749255713
0.014022609535786521
0.013828053896802674
0.013638584598469559
0.01345402391748481
0.01327420133260089
0.013098953225634117
0.012928122592241242
0.012761558762634951
